In [14]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
from tkinter import TclError

# use this backend to display in separate Tk window
%matplotlib tk

In [15]:
import pyaudio
import numpy
import struct

CHUNK=2048

def generateAudioChunks(**kwargs):
    '''
    params:
        RATE: Sampling rate for audio signals (8000)
        TIME: Time to record in seconds
    '''
    audio = pyaudio.PyAudio()

    stream = audio.open(format=pyaudio.paInt16, channels=1,
                        rate=kwargs["RATE"], input=True,
                        frames_per_buffer=CHUNK)

    while True:
        data = stream.read(CHUNK, exception_on_overflow=False)
        data_int = numpy.array(struct.unpack(
            str(2*CHUNK) + 'B', data))[::2]
        yield data_int

In [16]:
# create matplotlib figure and axes
fig, ax = plt.subplots(1, figsize=(15, 7))

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)

# create a line object with random data
line, = ax.plot(x, np.random.rand(CHUNK), '-', lw=2)

# basic formatting for the axes
ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('samples')
ax.set_ylabel('volume')
ax.set_ylim(0, 255)
ax.set_xlim(0, 2 * CHUNK)
plt.setp(ax, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# show the plot
plt.show(block=False)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

for data_int in generateAudioChunks(RATE=8000,TIME=10):
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b') + 128
    line.set_ydata(data_np)
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
    except TclError:
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

stream started
stream stopped
average frame rate = 4 FPS
